In [1]:
import win32com.client
import re
import time
import requests 


from splinter import Browser 
from bs4 import BeautifulSoup
import pandas as pd





In [2]:
my_outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

In [3]:
folder = my_outlook.GetDefaultFolder(6).Folders.Item("Lexology")
lex_urls = []
for item in folder.Items:    
    emailbody = item.body
    lexology_date = item.SentOn.strftime("%Y-%m-%d")
    lexology_url = re.findall(r'[<]\S*', emailbody)[0].strip('<').strip('>')
    print(lexology_url, lexology_date)
    lex_urls.append(lexology_url)
  

https://eur01.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.lexology.com%2Fnewsfeedemail%2Fhtml%3Fg%3Dcd0e6f28-edbf-49ae-894c-ac59699c5947%26d%3D2020-05-01&data=02%7C01%7C%7C214ba5a015b34e74d6ec08d7edbb5dc5%7C84df9e7fe9f640afb435aaaaaaaaaaaa%7C1%7C0%7C637239261301349941&sdata=u31kaeZReLe2Ugs2Du3vIVBXncFlESVLBMKS45dGopk%3D&reserved=0 2020-05-01
https://eur03.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.lexology.com%2Fnewsfeedemail%2Fhtml%3Fg%3Dcd0e6f28-edbf-49ae-894c-ac59699c5947%26d%3D2020-04-30&data=02%7C01%7C%7Ce2c06f5c44e54eb0ad0e08d7ecf1d7ff%7C84df9e7fe9f640afb435aaaaaaaaaaaa%7C1%7C0%7C637238395772752637&sdata=8qznFrzpas4ArNHhZvS5tRvx3dR8gdith6OnXjHFOLE%3D&reserved=0 2020-04-30


In [4]:
lex_urls

['https://eur01.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.lexology.com%2Fnewsfeedemail%2Fhtml%3Fg%3Dcd0e6f28-edbf-49ae-894c-ac59699c5947%26d%3D2020-05-01&data=02%7C01%7C%7C214ba5a015b34e74d6ec08d7edbb5dc5%7C84df9e7fe9f640afb435aaaaaaaaaaaa%7C1%7C0%7C637239261301349941&sdata=u31kaeZReLe2Ugs2Du3vIVBXncFlESVLBMKS45dGopk%3D&reserved=0',
 'https://eur03.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.lexology.com%2Fnewsfeedemail%2Fhtml%3Fg%3Dcd0e6f28-edbf-49ae-894c-ac59699c5947%26d%3D2020-04-30&data=02%7C01%7C%7Ce2c06f5c44e54eb0ad0e08d7ecf1d7ff%7C84df9e7fe9f640afb435aaaaaaaaaaaa%7C1%7C0%7C637238395772752637&sdata=8qznFrzpas4ArNHhZvS5tRvx3dR8gdith6OnXjHFOLE%3D&reserved=0']

In [5]:
import luigi
import pandas as pd

In [6]:
def get_article(article_link):
    r = requests.get(article_link) 
    html = r.text
    #time.sleep(5)

    #article title
    article_title_soup = BeautifulSoup(html, 'html.parser')
    #Soup_div = soup.find('div', class_='screenonly')
    article_title = article_title_soup.select('h1')[0]
    title = article_title.text.strip()

    # article date 
    Soup_date = article_title_soup.find('div', class_='article-attributes')
    article_date = Soup_date.find('span', class_='publication')
    article_date = article_date.text.strip()


    article_div = article_title_soup.find('div', class_='article-body')
    article = article_div.text.strip()
    return [title, article_date, article]
    

class DownloadArticle(luigi.Task):

    lexology_urls = luigi.ListParameter()# luigi parameter
    def run(self):
        article_file = []
        for lexology_url in self.lexology_urls:
            r = requests.get(lexology_url)
            html = r.text
            soup = BeautifulSoup(html, 'html.parser')
            article_header = soup.select('h4')
            article_link_list = []
            for links in article_header:
                article_links = links.find_all('a', href=True)
                for link in article_links: 
                    article_link = link.get('href')
                    article_link_list.append(article_link)
                    #print(article_link_list)
            text_dict = {}
            all_text = []
            for article_link in article_link_list:
                try: 
                    text = get_article(article_link)
                    all_text.append(text)
                except Exception as e:
                    print(f'error in {article_link}, {e}')

                
        df = pd.DataFrame(all_text, columns=['article_title','article_date','text'])
        df.to_pickle(self.output().open('w').path)
    
    def output(self):
        return luigi.LocalTarget(f'data/raw/raw_articles.pkl')

In [7]:
lexology_url, lexology_date

('https://eur03.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.lexology.com%2Fnewsfeedemail%2Fhtml%3Fg%3Dcd0e6f28-edbf-49ae-894c-ac59699c5947%26d%3D2020-04-30&data=02%7C01%7C%7Ce2c06f5c44e54eb0ad0e08d7ecf1d7ff%7C84df9e7fe9f640afb435aaaaaaaaaaaa%7C1%7C0%7C637238395772752637&sdata=8qznFrzpas4ArNHhZvS5tRvx3dR8gdith6OnXjHFOLE%3D&reserved=0',
 '2020-04-30')

In [8]:
luigi.build([DownloadArticle(lexology_urls=lex_urls)], local_scheduler=True)

DEBUG: Checking if DownloadArticle(lexology_urls=["https://eur01.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.lexology.com%2Fnewsfeedemail%2Fhtml%3Fg%3Dcd0e6f28-edbf-49ae-894c-ac59699c5947%26d%3D2020-05-01&data=02%7C01%7C%7C214ba5a015b34e74d6ec08d7edbb5dc5%7C84df9e7fe9f640afb435aaaaaaaaaaaa%7C1%7C0%7C637239261301349941&sdata=u31kaeZReLe2Ugs2Du3vIVBXncFlESVLBMKS45dGopk%3D&reserved=0", "https://eur03.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.lexology.com%2Fnewsfeedemail%2Fhtml%3Fg%3Dcd0e6f28-edbf-49ae-894c-ac59699c5947%26d%3D2020-04-30&data=02%7C01%7C%7Ce2c06f5c44e54eb0ad0e08d7ecf1d7ff%7C84df9e7fe9f640afb435aaaaaaaaaaaa%7C1%7C0%7C637238395772752637&sdata=8qznFrzpas4ArNHhZvS5tRvx3dR8gdith6OnXjHFOLE%3D&reserved=0"]) is complete
INFO: Informed scheduler that task   DownloadArticle___https___eur01__4b8b2aaa83   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 

error in https://www.lexology.com/r.ashx?i=7169559&l=8X3MHP2, 'NoneType' object has no attribute 'find'
error in https://www.lexology.com/r.ashx?i=7169559&l=8X3MHTN, 'NoneType' object has no attribute 'find'
error in https://www.lexology.com/r.ashx?i=7169559&l=8X5ADET, 'NoneType' object has no attribute 'find'
error in https://www.lexology.com/r.ashx?i=7169559&l=8X5E8JQ, 'NoneType' object has no attribute 'find'
error in https://www.lexology.com/r.ashx?i=7169559&l=8X41K8A, 'NoneType' object has no attribute 'find'
error in https://www.lexology.com/r.ashx?i=7169559&l=8X41K8K, 'NoneType' object has no attribute 'find'
error in https://www.lexology.com/r.ashx?i=7169559&l=8X41K8N, 'NoneType' object has no attribute 'find'
error in https://www.lexology.com/r.ashx?i=7169559&l=8X41K8R, 'NoneType' object has no attribute 'find'


Exception ignored in: <function TextWrapper.__del__ at 0x000002D1F8E72488>
Traceback (most recent call last):
  File "C:\Users\Carol\AppData\Roaming\Python\Python37\site-packages\luigi\format.py", line 415, in __del__
    self._stream.__del__(*args)
  File "C:\Users\Carol\AppData\Roaming\Python\Python37\site-packages\luigi\target.py", line 340, in __del__
    os.remove(self.tmp_path)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'data/raw/raw_articles.pkl-luigi-tmp-3801027528'
Exception ignored in: <function AtomicLocalFile.__del__ at 0x000002D1F8E63510>
Traceback (most recent call last):
  File "C:\Users\Carol\AppData\Roaming\Python\Python37\site-packages\luigi\target.py", line 340, in __del__
    os.remove(self.tmp_path)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'data/raw/raw_articles.pkl-luigi-tmp-3801027528'
INFO: [pid 14644] Worker Worker(salt=55944

True

In [9]:
pd.read_pickle('../data/raw/raw_articles.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/raw_articles.pkl'

In [ ]:
class PrepareData(luigi.Task):

    lexology_url = luigi.ListParameter()

    def requires(self):
        return DownloadArticle(lexology_urls=lexology_ursl)
        
    def run(self):
        df = pd.read_pickle(self.input().path)
        
        #Save final result.
        #df.to_pickle(self.output().open('w').path)
    
    def output(self):
        return luigi.LocalTarget(f'data/clean/clean_articles.pkl')